<img width="8%" alt="OWID.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/OWID.png" style="border-radius: 15%">

# OWID - Visualize GDP per capita through the years
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=OWID+-+Visualize+GDP+per+capita+through+the+years:+Error+short+description">Bug report</a>

**Tags:** #dash #dashboard #plotly #naas #asset #analytics #dropdown #callback #bootstrap #snippet

**Author:** [Zihui Ouyang](https://www.linkedin.com/in/zihui-ouyang-539626227/)

**Last update:** 2023-07-25 (Created: 2023-07-25)

**Description:** This notebook creates an interactive plot using Dash app infrastructure with OWID's GDP per capita data. The values are calculated by taking the expenditure-side real GDP at chained PPPs and dividing by the population.

**References:**
- https://www.rug.nl/ggdc/productivity/pwt/?lang=en
- https://stackoverflow.com/questions/70886359/dash-python-making-subplots-when-multiple-parameters-are-selected
- https://dash-example-index.herokuapp.com/line-charts

## Input

### Import libraries

In [ ]:
try:
    import dash
    import os
except:
    !pip install dash --user
    import dash
try:
    import dash_bootstrap_components as dbc
except:
    !pip install dash_bootstrap_components --user
    import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px

### Setup Variables
- `DASH_PORT`: specify a port number for Dash
- `url`: URL to get data from Excel
- `title`: App title

In [ ]:
DASH_PORT = 8050
url = "https://dataverse.nl/api/access/datafile/354095"
title = "GDP per capita"

## Model

### Initialize Dash app
The `os.environ.get("JUPYTERHUB_USER")` is used to access the environment variable `JUPYTERHUB_USER` already stored into your Naas Lab.

In [ ]:
app = dash.Dash(
    title=title,
    requests_pathname_prefix=f'/user/{os.environ.get("JUPYTERHUB_USER")}/proxy/{DASH_PORT}/',
    external_stylesheets=[dbc.themes.BOOTSTRAP],
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1.0"}
    ],
)

#app = dash.Dash(title = "GDP per capita") if you are not in Naas

### Get Data

In [ ]:
contents_df = pd.read_excel(url, sheet_name="Data")
new_column = []
for i in range(len(contents_df)):
    if contents_df.iloc[i,4] == np.nan:
        new_column.append(0)
    else:
        new_column.append(contents_df.iloc[i,4]/contents_df.iloc[i,6])
new_dict = {"Country": contents_df["country"],
           "Year": contents_df["year"],
           "gdp": new_column}
new_df = pd.DataFrame(data=new_dict)

### Create Dash App

In [ ]:
p = "gdp"
app.layout = html.Div(
    [
        html.H4("GDP per capita from 1950 to 2019"),
        html.P("Select countries"),
        dcc.Dropdown(
            id="countries",
            options=new_df.Country.unique(),
            value=["United States"],
            multi = True
        ),
        dcc.RangeSlider(id='slider', min=1950, max=2019, value=[1950, 2019],
               marks={x: str(x) for x in [1950, 1965, 1980, 1995, 2010, 2019]}),
        dcc.Graph(id=p, figure={}, style={'display': 'none'})
    ]
)

@app.callback(
    Output(p, 'figure'),
    Output(p, 'style'),
    Input('countries', 'value'),
    Input('slider', 'value')
)

def update_graph(country, year):
    dff = (new_df["Country"].isin(country)) & (new_df["Year"] <= year[1]) & (new_df["Year"] >= year[0])
    figures = px.line(new_df[dff], x='Year', y=p, color = "Country", markers = True).update_layout(
                                plot_bgcolor='rgba(0, 0, 0, 0)', height= 600
                            )
    styles = {'display': 'block'}   
    return figures, styles

## Output

### Generate URL and show logs

In [ ]:
if __name__ == "__main__":
    app.run_server(proxy=f"http://127.0.0.1:{DASH_PORT}::https://app.naas.ai")